# 🚀 RL Football - GPU Training v2

**Optimized for T4 GPU with 64 parallel environments**

1. Runtime → GPU (T4) ✓
2. Run all cells
3. Download `trained.json` when done

**Expected: 100k episodes in ~2-4 hours**

In [ ]:
# Setup
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json, time

print(f'TensorFlow: {tf.__version__}')
gpus = tf.config.list_physical_devices('GPU')
print(f'GPU: {gpus}')
if not gpus:
    print('⚠️ No GPU! Go to Runtime → Change runtime type → T4 GPU')
else:
    print('✅ GPU ready')

In [ ]:
# ============================================================
# CONFIGURATION - Adjust these for your needs
# ============================================================
EPISODES = 100000     # Total episodes to train
N_ENVS = 64           # Parallel environments (64 is good for T4)
BATCH_SIZE = 128      # Larger batch = better GPU utilization
TRAIN_EVERY = 2       # Train every N steps (lower = more training)
SAVE_EVERY = 10000    # Save checkpoint every N episodes

# Hyperparameters (optimized)
GAMMA = 0.995
LR = 0.0005
EPS_DECAY = 0.9999
EPS_MIN = 0.02
TARGET_UPDATE = 500
BUFFER_SIZE = 100000

print(f'Config: {EPISODES} episodes, {N_ENVS} parallel envs, batch {BATCH_SIZE}')

In [ ]:
# ============================================================
# VECTORIZED GAME ENGINE
# ============================================================
class Game:
    """Runs N football games in parallel using pure NumPy."""
    
    def __init__(self, n):
        self.n = n
        self.W, self.H = 720, 420
        # Pre-compute movement vectors
        self.MOVES = np.array([
            [0,-1], [0,1], [-1,0], [1,0],
            [-1,-1], [1,-1], [-1,1], [1,1],
            [0,0], [0,0]
        ], dtype=np.float32)
        self.reset()
    
    def reset(self):
        """Reset all environments."""
        self.p1 = np.zeros((self.n, 4), dtype=np.float32)  # x, y, vx, vy
        self.p2 = np.zeros((self.n, 4), dtype=np.float32)
        self.ball = np.zeros((self.n, 4), dtype=np.float32)
        self.p1[:, :2] = [120, 210]
        self.p2[:, :2] = [600, 210]
        self.ball[:, :2] = [360, 210]
        self.scores = np.zeros((self.n, 2), dtype=np.int32)
        self.time = np.full(self.n, 30.0, dtype=np.float32)
        self.done = np.zeros(self.n, dtype=bool)
        self.kick = np.zeros((self.n, 2), dtype=bool)
    
    def step(self, a1, a2):
        """Step all environments. Returns events and done flags."""
        # Apply movement (fully vectorized)
        m1, m2 = self.MOVES[a1], self.MOVES[a2]
        self.p1[:, 2:4] += m1 * 2
        self.p2[:, 2:4] += m2 * 2
        
        # Record kicks
        self.kick[:, 0] = (a1 == 8)
        self.kick[:, 1] = (a2 == 8)
        
        # Clamp speeds
        for p in [self.p1, self.p2]:
            speed = np.sqrt(p[:, 2]**2 + p[:, 3]**2)
            fast = speed > 4
            if np.any(fast):
                scale = 4.0 / np.maximum(speed[fast], 1e-8)
                p[fast, 2] *= scale
                p[fast, 3] *= scale
        
        # Update positions
        self.p1[:, :2] += self.p1[:, 2:4]
        self.p2[:, :2] += self.p2[:, 2:4]
        self.ball[:, :2] += self.ball[:, 2:4]
        
        # Apply friction
        self.p1[:, 2:4] *= 0.85
        self.p2[:, 2:4] *= 0.85
        self.ball[:, 2:4] *= 0.98
        
        # Constrain players
        self.p1[:, 0] = np.clip(self.p1[:, 0], 25, self.W - 25)
        self.p1[:, 1] = np.clip(self.p1[:, 1], 25, self.H - 25)
        self.p2[:, 0] = np.clip(self.p2[:, 0], 25, self.W - 25)
        self.p2[:, 1] = np.clip(self.p2[:, 1], 25, self.H - 25)
        
        # Ball wall bounces
        top = self.ball[:, 1] < 12
        bot = self.ball[:, 1] > self.H - 12
        self.ball[top, 1] = 12
        self.ball[bot, 1] = self.H - 12
        self.ball[top | bot, 3] *= -0.8
        
        # Side walls (outside goal area)
        in_goal = (self.ball[:, 1] > 150) & (self.ball[:, 1] < 270)
        left = (self.ball[:, 0] < 12) & ~in_goal
        right = (self.ball[:, 0] > self.W - 12) & ~in_goal
        self.ball[left, 0] = 12
        self.ball[right, 0] = self.W - 12
        self.ball[left | right, 2] *= -0.8
        
        # Player-ball collisions (vectorized)
        for i, p in enumerate([self.p1, self.p2]):
            dx = self.ball[:, 0] - p[:, 0]
            dy = self.ball[:, 1] - p[:, 1]
            dist = np.sqrt(dx**2 + dy**2)
            hit = (dist > 0) & (dist < 37)
            if np.any(hit):
                nx = dx[hit] / dist[hit]
                ny = dy[hit] / dist[hit]
                self.ball[hit, 0] = p[hit, 0] + nx * 37
                self.ball[hit, 1] = p[hit, 1] + ny * 37
                power = np.where(self.kick[hit, i], 12.0, 6.0)
                self.ball[hit, 2] = nx * power + p[hit, 2] * 0.5
                self.ball[hit, 3] = ny * power + p[hit, 3] * 0.5
        
        # Check goals
        events = np.zeros(self.n, dtype=np.int32)  # 0=none, 1=p1 scored, -1=p2 scored
        
        p2_goal = in_goal & (self.ball[:, 0] < 0)
        p1_goal = in_goal & (self.ball[:, 0] > self.W)
        
        if np.any(p2_goal):
            self.scores[p2_goal, 1] += 1
            events[p2_goal] = -1
            self._reset_pos(p2_goal)
        
        if np.any(p1_goal):
            self.scores[p1_goal, 0] += 1
            events[p1_goal] = 1
            self._reset_pos(p1_goal)
        
        # Update time
        self.time -= 1/60
        self.done = self.time <= 0
        
        return events, self.done.copy()
    
    def _reset_pos(self, mask):
        self.p1[mask, :2] = [120, 210]
        self.p2[mask, :2] = [600, 210]
        self.ball[mask] = [360, 210, 0, 0]
    
    def get_state(self, player=1):
        """Get observation for player 1 or 2."""
        if player == 1:
            p, o = self.p1, self.p2
        else:
            p, o = self.p2, self.p1
        
        dist = np.sqrt((p[:, 0] - self.ball[:, 0])**2 + 
                      (p[:, 1] - self.ball[:, 1])**2) / 830
        
        return np.column_stack([
            p[:, 0] / self.W,
            p[:, 1] / self.H,
            self.ball[:, 0] / self.W,
            self.ball[:, 1] / self.H,
            np.clip(self.ball[:, 2] / 15, -1, 1),
            np.clip(self.ball[:, 3] / 15, -1, 1),
            o[:, 0] / self.W,
            o[:, 1] / self.H,
            dist,
            np.zeros(self.n),  # angle placeholder
            np.zeros(self.n),  # goal dist placeholder
            np.zeros(self.n),  # goal angle placeholder
        ]).astype(np.float32)

print('✅ Game engine ready')

In [ ]:
# ============================================================
# SIMPLE AI OPPONENT (fully vectorized)
# ============================================================
def ai_actions(states):
    """Vectorized AI that chases ball and kicks when close."""
    n = len(states)
    px, py = states[:, 0], states[:, 1]
    bx, by = states[:, 2], states[:, 3]
    dist = states[:, 8]
    
    # Default: stay
    actions = np.full(n, 9, dtype=np.int32)
    
    # Kick when very close
    actions[dist < 0.04] = 8
    
    # Chase ball otherwise
    dx = bx - px
    dy = by - py
    
    # Encode movement as single number for vectorized lookup
    mx = (dx > 0.02).astype(np.int32) - (dx < -0.02).astype(np.int32)  # 1, 0, -1
    my = (dy > 0.02).astype(np.int32) - (dy < -0.02).astype(np.int32)  # 1, 0, -1
    
    # Map (mx, my) to action index
    # (0,-1)->0, (0,1)->1, (-1,0)->2, (1,0)->3
    # (-1,-1)->4, (1,-1)->5, (-1,1)->6, (1,1)->7
    chase = dist >= 0.04
    
    actions[chase & (mx == 0) & (my == -1)] = 0
    actions[chase & (mx == 0) & (my == 1)] = 1
    actions[chase & (mx == -1) & (my == 0)] = 2
    actions[chase & (mx == 1) & (my == 0)] = 3
    actions[chase & (mx == -1) & (my == -1)] = 4
    actions[chase & (mx == 1) & (my == -1)] = 5
    actions[chase & (mx == -1) & (my == 1)] = 6
    actions[chase & (mx == 1) & (my == 1)] = 7
    
    return actions

print('✅ AI ready')

In [ ]:
# ============================================================
# DUELING DQN MODEL
# ============================================================
def create_model():
    """Create Dueling DQN matching browser architecture."""
    inp = layers.Input(shape=(12,))
    x = layers.Dense(256, activation='relu', kernel_initializer='he_normal')(inp)
    x = layers.Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = layers.Dense(128, activation='relu', kernel_initializer='he_normal')(x)
    
    # Value stream
    v = layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    v = layers.Dense(1, kernel_initializer='he_normal')(v)
    
    # Advantage stream
    a = layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    a = layers.Dense(10, kernel_initializer='he_normal')(a)
    
    # Combine: Q = V + (A - mean(A))
    mean_a = layers.Lambda(lambda t: tf.reduce_mean(t, axis=1, keepdims=True))(a)
    q = layers.Add()([v, layers.Subtract()([a, mean_a])])
    
    model = keras.Model(inp, q)
    return model

# Compiled functions for GPU efficiency
@tf.function(reduce_retracing=True)
def predict_q(model, states):
    return model(states, training=False)

@tf.function(reduce_retracing=True)
def train_step(model, optimizer, states, targets):
    with tf.GradientTape() as tape:
        q = model(states, training=True)
        loss = tf.reduce_mean(tf.square(q - targets))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

print('✅ Model ready')

In [ ]:
# ============================================================
# EFFICIENT REPLAY BUFFER (NumPy-based)
# ============================================================
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.states = np.zeros((capacity, 12), dtype=np.float32)
        self.actions = np.zeros(capacity, dtype=np.int32)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.next_states = np.zeros((capacity, 12), dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=bool)
        self.ptr = 0
        self.size = 0
    
    def add(self, s, a, r, s2, d):
        """Add batch of transitions."""
        n = len(s)
        idx = np.arange(self.ptr, self.ptr + n) % self.capacity
        self.states[idx] = s
        self.actions[idx] = a
        self.rewards[idx] = r
        self.next_states[idx] = s2
        self.dones[idx] = d
        self.ptr = (self.ptr + n) % self.capacity
        self.size = min(self.size + n, self.capacity)
    
    def sample(self, batch_size):
        idx = np.random.choice(self.size, batch_size, replace=False)
        return (self.states[idx], self.actions[idx], self.rewards[idx],
                self.next_states[idx], self.dones[idx])

print('✅ Buffer ready')

In [ ]:
# ============================================================
# REWARD FUNCTION (vectorized)
# ============================================================
def compute_rewards(game, events, prev_dist):
    """Calculate rewards for all environments."""
    n = game.n
    p, b, o = game.p1, game.ball, game.p2
    
    dist = np.sqrt((p[:, 0] - b[:, 0])**2 + (p[:, 1] - b[:, 1])**2)
    r = np.zeros(n, dtype=np.float32)
    
    # Goal rewards
    r[events == 1] += 500   # Scored
    r[events == -1] -= 300  # Conceded
    
    # Proximity reward (0-5 based on distance)
    r += (1 - dist / 830) * 5
    
    # Ball possession bonus
    r[dist < 40] += 10
    
    # Approach reward
    if prev_dist is not None:
        delta = prev_dist - dist
        r += delta * 0.5
        r[delta > 2] += 3  # Fast approach bonus
    
    # Movement rewards
    speed = np.sqrt(p[:, 2]**2 + p[:, 3]**2)
    r[(speed < 0.5) & (dist > 50)] -= 8  # Standing still penalty
    r[speed > 1] += 1  # Movement bonus
    
    # Attack rewards
    r[(b[:, 2] > 2) & (dist < 80)] += 8  # Ball moving toward goal
    r[np.abs(b[:, 0] - 720) < 100] += 5  # Ball near goal
    
    # Penalties
    corner = ((p[:, 0] < 80) | (p[:, 0] > 640)) & ((p[:, 1] < 80) | (p[:, 1] > 340))
    r[corner] -= 5
    r[corner & (dist > 100)] -= 5
    
    r[dist > 300] -= 5
    r[(dist > 200) & (dist <= 300)] -= 3
    r[(dist > 150) & (dist <= 200)] -= 1
    
    # Opponent closer penalty
    o_dist = np.sqrt((o[:, 0] - b[:, 0])**2 + (o[:, 1] - b[:, 1])**2)
    r[(o_dist < dist) & (dist > 60)] -= 2
    
    r -= 0.1  # Time penalty
    
    return r, dist

print('✅ Rewards ready')

In [ ]:
# ============================================================
# MAIN TRAINING LOOP
# ============================================================
def train():
    print('=' * 60)
    print('🚀 RL Football GPU Training')
    print('=' * 60)
    
    # Initialize
    game = Game(N_ENVS)
    model = create_model()
    target = create_model()
    target.set_weights(model.get_weights())
    optimizer = keras.optimizers.Adam(LR)
    buffer = ReplayBuffer(BUFFER_SIZE)
    
    eps = 1.0
    train_steps = 0
    total_ep = 0
    step_count = 0
    stats = {'W': 0, 'L': 0, 'D': 0, 'goals': 0}
    t0 = time.time()
    
    print(f'Training {EPISODES} episodes with {N_ENVS} parallel envs...')
    print('=' * 60)
    
    while total_ep < EPISODES:
        game.reset()
        prev_dist = None
        
        while not np.all(game.done):
            step_count += 1
            active = ~game.done
            
            # Get states
            s1 = game.get_state(1)
            s2 = game.get_state(2)
            
            # Select actions
            a1 = np.random.randint(0, 10, N_ENVS)  # Start with random
            exploit = np.random.random(N_ENVS) >= eps
            if np.any(exploit):
                q = predict_q(model, tf.constant(s1[exploit])).numpy()
                a1[exploit] = np.argmax(q, axis=1)
            
            a2 = ai_actions(s2)
            
            # Step environment
            events, dones = game.step(a1, a2)
            
            # Calculate rewards
            rewards, prev_dist = compute_rewards(game, events, prev_dist)
            
            # Get next states
            next_s1 = game.get_state(1)
            
            # Store transitions (only active envs)
            buffer.add(s1[active], a1[active], rewards[active], 
                       next_s1[active], dones[active])
            
            # Track goals
            stats['goals'] += np.sum(events != 0)
            
            # Training step
            if buffer.size >= 1000 and step_count % TRAIN_EVERY == 0:
                s, a, r, s2, d = buffer.sample(BATCH_SIZE)
                
                # Double DQN target calculation
                q_next = predict_q(model, tf.constant(s2)).numpy()
                q_target = predict_q(target, tf.constant(s2)).numpy()
                best_a = np.argmax(q_next, axis=1)
                next_q = q_target[np.arange(BATCH_SIZE), best_a]
                
                # Build targets
                targets = predict_q(model, tf.constant(s)).numpy()
                targets[np.arange(BATCH_SIZE), a] = r + GAMMA * next_q * ~d
                
                # Train
                train_step(model, optimizer, tf.constant(s), tf.constant(targets))
                train_steps += 1
                
                # Update target network
                if train_steps % TARGET_UPDATE == 0:
                    target.set_weights(model.get_weights())
        
        # Episode complete - count outcomes
        for i in range(N_ENVS):
            if game.scores[i, 0] > game.scores[i, 1]:
                stats['W'] += 1
            elif game.scores[i, 1] > game.scores[i, 0]:
                stats['L'] += 1
            else:
                stats['D'] += 1
        
        total_ep += N_ENVS
        
        # Decay epsilon
        eps = max(EPS_MIN, eps * (EPS_DECAY ** N_ENVS))
        
        # Progress logging
        if total_ep % (50 * N_ENVS) < N_ENVS:
            elapsed = time.time() - t0
            eps_rate = total_ep / elapsed if elapsed > 0 else 0
            eta = (EPISODES - total_ep) / eps_rate if eps_rate > 0 else 0
            eta_str = f'{eta/3600:.1f}h' if eta > 3600 else f'{eta/60:.0f}m'
            print(f'Ep {total_ep:,}/{EPISODES:,} | ε:{eps:.3f} | '
                  f'W:{stats["W"]} L:{stats["L"]} D:{stats["D"]} | '
                  f'{eps_rate:.1f}/s | ETA:{eta_str}')
        
        # Save checkpoint
        if total_ep % SAVE_EVERY < N_ENVS:
            save_weights(model, eps, train_steps, f'weights_{total_ep}.json')
    
    # Final save
    save_weights(model, eps, train_steps, 'trained.json')
    
    print('=' * 60)
    print('✅ Training Complete!')
    print(f'Final: W:{stats["W"]} L:{stats["L"]} D:{stats["D"]}')
    print(f'Total goals: {stats["goals"]}')
    print(f'Time: {(time.time() - t0) / 3600:.2f} hours')
    print('=' * 60)
    
    return model, stats

def save_weights(model, eps, steps, filename):
    weights = [{'shape': list(w.shape), 'data': w.flatten().tolist()} 
               for w in model.get_weights()]
    agent = {'weights': weights, 'epsilon': float(eps), 'trainStepCount': int(steps)}
    data = {
        'version': 2,
        'aiType': 'dqn',
        'blipAgent': agent,
        'bloopAgent': agent,
        'blip': agent,
        'bloop': agent
    }
    with open(filename, 'w') as f:
        json.dump(data, f)
    print(f'💾 Saved {filename}')

print('✅ Training function ready - Run next cell to start!')

In [ ]:
# 🚀 START TRAINING
model, stats = train()

In [ ]:
# Download trained weights
from google.colab import files
files.download('trained.json')